In [3]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torch
import tensorflow as tf


In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

print(device)
# 데이터 로드 및 전처리
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=100, shuffle=True)
test_dataloader = DataLoader(test_dataset, shuffle=False)

# 모델 정의
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.linear1 = nn.Linear(28 * 28, 100)
        self.linear2 = nn.Linear(100, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.dropout(self.relu(self.linear1(x)))
        x = self.dropout(self.relu(self.linear2(x)))
        return x

model = CustomModel().to(device)

# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# 학습
num_epochs = 15

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_dataloader:

        inputs=inputs.to(device)
        labels=labels.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch % 1 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch, num_epochs, loss.item()))

# 테스트
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_dataloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print('Test Accuracy: {:.2%}'.format(accuracy))


cuda
Epoch [0/15], Loss: 1.3905
Epoch [1/15], Loss: 1.4977
Epoch [2/15], Loss: 1.6270
Epoch [3/15], Loss: 1.6448
Epoch [4/15], Loss: 1.2637
Epoch [5/15], Loss: 1.6447
Epoch [6/15], Loss: 1.4605
Epoch [7/15], Loss: 1.4500
Epoch [8/15], Loss: 1.4430
Epoch [9/15], Loss: 1.4375
Epoch [10/15], Loss: 1.4110
Epoch [11/15], Loss: 1.3112
Epoch [12/15], Loss: 1.4254
Epoch [13/15], Loss: 1.5319
Epoch [14/15], Loss: 1.6217


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

Epoch [0/10], Cost: 2.302562
Epoch [1/10], Cost: 2.302585
Epoch [2/10], Cost: 2.302648
Epoch [3/10], Cost: 2.302413
Epoch [4/10], Cost: 2.302533
Epoch [5/10], Cost: 2.302485
Epoch [6/10], Cost: 2.302483
Epoch [7/10], Cost: 2.302363
Epoch [8/10], Cost: 2.302352
Epoch [9/10], Cost: 2.302512
